## Answer to Question-2  starts here

In [2]:
#importing pandas 
import pandas as pd 
html = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df= pd.DataFrame(html[0])
dfclean = df[df['Borough']!='Not assigned']
dfclean.tail()

,Postcode,Borough,Neighbourhood
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


 #### Get the latitude and the longitude coordinates of each neighborhood.

In [3]:
geodf = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join both the datasets on postal code column to get teh corresponding latitude and longitude

In [5]:
 result = pd.merge(dfclean,geodf,left_on='Postcode',right_on='Postal Code',how='left')

In [6]:
result.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763


In [7]:
result = result.drop(axis = 1,columns=['Postal Code'])
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [8]:
result.shape

(210, 5)

# Answer to the question 3 starts here

#### Objective:
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

#### Selecting the Borough which has word Toronto in it

In [57]:
BoroughToronto=result[result['Borough'].str.contains('Toronto', case=False)].reset_index(drop=True)
BoroughToronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(result['Borough'].unique()), result.shape[0] ))
result.groupby(['Borough','Postcode']).count()
result['Borough'].value_counts()

The dataframe has 10 boroughs and 210 neighborhoods.


Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

#### Use geopy library to get the latitude and longitude values of Toronto City.

In [18]:
from geopy.geocoders import Nominatim

In [19]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(BoroughToronto['Latitude'], BoroughToronto['Longitude'], BoroughToronto['Borough'], BoroughToronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Create a forsquare query and find the top 100 venues in the neighbourhood

In [23]:
# The code was removed by Watson Studio for sharing.

In this cell added foursquare credentails data


In [46]:
neighbourhood_name = result.loc[1,'Neighbourhood']
neighbourhood_latitude =result.loc[1,'Latitude']
neighbourhood_longitude = result.loc[1,'Longitude']
print (neighbourhood_name,neighbourhood_latitude,neighbourhood_longitude)

Victoria Village 43.725882299999995 -79.31557159999998


In [62]:
neighbourhood_name = BoroughToronto.loc[1,'Neighbourhood']
neighbourhood_latitude =BoroughToronto.loc[1,'Latitude']
neighbourhood_longitude = BoroughToronto.loc[1,'Longitude']
print (neighbourhood_name,neighbourhood_latitude,neighbourhood_longitude)

Queen's Park 43.6623015 -79.3894938


In [63]:
# The code was removed by Watson Studio for sharing.

"https://api.foursquare.com/v2/venues/search?client_id=0VX3UDVRSCFN3HR3L1R32Q0BDFTNDZ02WIBGABHCON3XJJG5&client_secret=GZKEQTFMI4U0UNXIN4RYAK0EV1HSEASBPG4USXYYWCXUNZ3M&ll=43.6623015,-79.3894938&v=20180604&query=Queen's Park&radius=5000&limit=100"

#### Send the request and get the response for analysis

In [64]:
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
fsresults = requests.get(url).json()
fsresults

{'meta': {'code': 200, 'requestId': '5e5ecd6c760a7f001bfef879'},
 'response': {'venues': [{'id': '4b170e47f964a5208cc123e3',
    'name': "Queen's Park Subway Station",
    'location': {'address': '691 University Ave',
     'crossStreet': 'at College St',
     'lat': 43.660006357009195,
     'lng': -79.39027220226043,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.660006357009195,
       'lng': -79.39027220226043}],
     'distance': 263,
     'postalCode': 'M5G 2P1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['691 University Ave (at College St)',
      'Toronto ON M5G 2P1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1fd931735',
      'name': 'Metro Station',
      'pluralName': 'Metro Stations',
      'shortName': 'Metro',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/subway_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1583271415',

In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [65]:

Toronto_venues = getNearbyVenues(names=BoroughToronto['Neighbourhood'],
                                   latitudes=BoroughToronto['Latitude'],
                                   longitudes=BoroughToronto['Longitude']
                                  )


Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

In [66]:
print(Toronto_venues.shape)
Toronto_venues.head()

(3273, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [68]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,18,18,18,18,18,18
Berczy Park,57,57,57,57,57,57
Brockton,24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
CN Tower,18,18,18,18,18,18
Cabbagetown,42,42,42,42,42,42
Central Bay Street,80,80,80,80,80,80
Chinatown,90,90,90,90,90,90


In [69]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.
